In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import psycopg2

Is there a statistical difference between the ratings of two different music genres? 
Is there a difference between the ratings of {insert genre here} music and all other music?
Are the albums from one label rated differently than the wider population?

In [35]:
#create connection
conn = psycopg2.connect("dbname=pitchfork_reviews")

In [36]:
cur = conn.cursor()

In [4]:
#execute simple commands
cur.execute("""
SELECT * FROM pg_catalog.pg_tables;
""")

In [5]:
i = cur.fetchall()
#load cast genres into a dataframe
df = pd.DataFrame(i)

In [6]:
df

,0,1,2,3,4,5,6,7
0,public,reviews,lee,None,False,False,False,False
1,public,artists,lee,None,False,False,False,False
2,public,genres,lee,None,False,False,False,False
3,public,labels,lee,None,False,False,False,False
4,public,years,lee,None,False,False,False,False
5,public,content,lee,None,False,False,False,False
6,pg_catalog,pg_statistic,s.aguiar,None,True,False,False,False
7,pg_catalog,pg_foreign_table,s.aguiar,None,True,False,False,False
8,pg_catalog,pg_authid,s.aguiar,pg_global,True,False,False,False
9,pg_catalog,pg_user_mapping,s.aguiar,None,True,False,False,False


In [7]:
#execute simple commands
cur.execute("""
SELECT * FROM genres;
""")

In [9]:
i = cur.fetchall()
#load cast genres into a dataframe
df = pd.DataFrame(i)

In [10]:
#check unique values in genres
df[1].unique()

array(['electronic', 'metal', 'rock', None, 'rap', 'experimental',
       'pop/r&b', 'folk/country', 'jazz', 'global'], dtype=object)

In [ ]:
df_genres.head(10)

In [ ]:
df_genres.info()

In [15]:
#execute simple commands
reviews = cur.execute("""
SELECT * FROM reviews;
""")

# SELECT c1, aggregate(c2)
# FROM t
# GROUP BY c1;

In [16]:
reviews = cur.fetchall()
#load cast genres into a dataframe
df_reviews = pd.DataFrame(reviews)

In [17]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18393 entries, 0 to 18392
Data columns (total 13 columns):
0     18393 non-null int64
1     18393 non-null object
2     18393 non-null object
3     18393 non-null object
4     18393 non-null float64
5     18393 non-null int64
6     18393 non-null object
7     14487 non-null object
8     18393 non-null object
9     18393 non-null int64
10    18393 non-null int64
11    18393 non-null int64
12    18393 non-null int64
dtypes: float64(1), int64(6), object(6)
memory usage: 1.8+ MB


In [18]:
df_reviews.head(20)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,22703,mezzanine,massive attack,http://pitchfork.com/reviews/albums/22703-mezz...,9.3,0,nate patrin,contributor,2017-01-08,6,8,1,2017
1,22721,prelapsarian,krallice,http://pitchfork.com/reviews/albums/22721-prel...,7.9,0,zoe camp,contributor,2017-01-07,5,7,1,2017
2,22659,all of them naturals,uranium club,http://pitchfork.com/reviews/albums/22659-all-...,7.3,0,david glickman,contributor,2017-01-07,5,7,1,2017
3,22661,first songs,"kleenex, liliput",http://pitchfork.com/reviews/albums/22661-firs...,9.0,1,jenn pelly,associate reviews editor,2017-01-06,4,6,1,2017
4,22725,new start,taso,http://pitchfork.com/reviews/albums/22725-new-...,8.1,0,kevin lozano,tracks coordinator,2017-01-06,4,6,1,2017
5,22722,insecure (music from the hbo original series),various artists,http://pitchfork.com/reviews/albums/22722-inse...,7.4,0,vanessa okoth-obbo,contributor,2017-01-05,3,5,1,2017
6,22704,stillness in wonderland,little simz,http://pitchfork.com/reviews/albums/22704-litt...,7.1,0,katherine st. asaph,contributor,2017-01-05,3,5,1,2017
7,22694,tehillim,yotam avni,http://pitchfork.com/reviews/albums/22694-tehi...,7.0,0,andy beta,contributor,2017-01-05,3,5,1,2017
8,22714,reflection,brian eno,http://pitchfork.com/reviews/albums/22714-refl...,7.7,0,andy beta,contributor,2017-01-04,2,4,1,2017
9,22724,filthy america its beautiful,the lox,http://pitchfork.com/reviews/albums/22724-filt...,5.3,0,ian cohen,contributor,2017-01-04,2,4,1,2017


In [37]:
#execute simple commands
query = cur.execute("""
SELECT 4 FROM reviews GROUP BY 6;
""")

InvalidColumnReference: GROUP BY position 6 is not in select list
LINE 2: SELECT 4, 6 FROM reviews GROUP BY 6;
                                          ^


In [33]:
fetch = cur.fetchall()
#load cast genres into a dataframe
df_query = pd.DataFrame(fetch)

In [34]:
df_query.head(10)

,0,1
0,4,6
1,4,6
2,4,6
3,4,6
4,4,6
5,4,6
6,4,6
7,4,6
8,4,6
9,4,6


In [ ]:
#### drop duplicates

# df.drop_duplicates(subset = 'id', keep='first', inplace=True)

In [ ]:
#execute simple commands
genres = cur.execute("""
SELECT reviews.reviewid, reviews.score, genres.genre FROM reviews
INNER JOIN genres ON reviews.reviewid = genres.reviewid
WHERE genres.genre = 'metal' OR genres.genre = 'jazz';
""")

In [ ]:
reviews_genres_inner = cur.fetchall()
#load cast genres into a dataframe
df_genre_reviews = pd.DataFrame(reviews_genres_inner)

In [ ]:
#### drop duplicates

# df_genre_reviews.drop_duplicates(subset = 'id', keep='first', inplace=True)

In [ ]:
df_genre_reviews.info()

In [ ]:
df_genre_reviews.head(5)

In [ ]:
df_genre_reviews.columns = ['reviewid', 'score', 'genre']

In [ ]:
df_genre_reviews.groupby(['genre']).mean()

#### 

In [ ]:
#execute simple commands
data = cur.execute("""
SELECT reviews.reviewid, reviews.score, genres.genre FROM reviews
INNER JOIN genres ON reviews.reviewid = genres.reviewid;
""")

In [ ]:
data = cur.fetchall()
#load cast genres into a dataframe
df_data = pd.DataFrame(data)

In [ ]:
df_data.describe(), df_data.info()

In [ ]:
df_data.head()

In [ ]:
df_data.columns = ['reviewid', 'score', 'genre']

In [ ]:
df_data.head()

In [ ]:
# summary statistics for score (dep var)
df_data.score.describe()

In [ ]:
df_data.head(10)

In [ ]:
score = df_data['score']

In [ ]:
log_score = np.log(df_data['score'])

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(8,8))
_ = ax[0].hist(score, bins=50)
# _ = ax[1].hist(log_score, bins=50)

In [ ]:
df_genre_reviews.describe()

In [ ]:
# query with an f-string
query = f"""
SELECT {', '.join(columns)} FROM dogs;
"""

cur.execute(query)

In [ ]:
# option 1 to display, using fetchall method
data = cur.fetchall()
data

In [ ]:
# option 1 to display, using fetchall method
temp_data = cur.fetchall()
temp_data

In [ ]:
# query with an f-string
query = f"""
SELECT * FROM genres;
"""

cur.execute(query)


In [ ]:
temp_data = cur.fetchall()
temp_data